# Executive Summary

Partnering up with the City of Ames, we analyse homes sold between 2006 to 2010. Through exploratory data analysis, we have found several outliers in some of the features and would have been excluded from our model to increase the models accuracy.

Pandas,Sklearn and Statistics Models are some of the tools that have been used in the exploratory data analysis and model predictions and derive at coefficients for each of the features to explain their impact on prices. During the exploration of the data, we encountered missing values that had to be dealt with. We have decided to most replace missing values with 0 or by replacing them with mean and medians or other forms of inferential data imputing methods. We also converted some of the data into numerical form and one hot encoded them, to aid in the modeling process.

We used a total of 4 models(linear regression, Ridge, Lasso and ElasticNet), a recursive feature elimination was done to retrieve the best 30 features that would predict the house prices. We use R2 and Square Root mean errors to compare the performance of the model to select the best performer. We then used that best model type to make predictions on our test dataset.


# Problem Statment

We are a team of Data Scientists engaged by the City of Ames to help predict the prices of houses based on their characteristics. The team would be using linear regression models based on data given by the City to predict housing sale prices. 

This analysis and prediction would help potential buyers and sellers by giving them a better understanding of the property market, enabling them to make better informed decisions.(i.e. Buying undervalued houses and renovating them to sell them for profit)

In this project, there will be several linear regression techniques such as Linear Regression, Lasso Regression, Ridge Regression and ElasticNet Regression. Recursive feature elimination will also be deployed to identify the top 30 features that are the best predictors of house prices in the City of Ames.  We will be using R2 and Square Room error to compare the performance of the different models.



In [1]:
#imports
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#reading csv
test=pd.read_csv("../datasets/test.csv")
train=pd.read_csv("../datasets/train.csv")


# Cleaning Train dataset

In [3]:
#exploring train 
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [4]:
train.shape

(2051, 81)

In [5]:
train.dtypes

Id                int64
PID               int64
MS SubClass       int64
MS Zoning        object
Lot Frontage    float64
                 ...   
Misc Val          int64
Mo Sold           int64
Yr Sold           int64
Sale Type        object
SalePrice         int64
Length: 81, dtype: object

In [6]:
train.columns


Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [7]:
# checking for null values in train 
train.isnull().sum().sort_values().tail(30)



Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Overall Cond         0
BsmtFin SF 1         1
Total Bsmt SF        1
BsmtFin SF 2         1
Bsmt Unf SF          1
Garage Area          1
Garage Cars          1
Bsmt Full Bath       2
Bsmt Half Bath       2
Mas Vnr Area        22
Mas Vnr Type        22
Bsmt Qual           55
Bsmt Cond           55
BsmtFin Type 1      55
BsmtFin Type 2      56
Bsmt Exposure       58
Garage Type        113
Garage Yr Blt      114
Garage Qual        114
Garage Cond        114
Garage Finish      114
Lot Frontage       330
Fireplace Qu      1000
Fence             1651
Alley             1911
Misc Feature      1986
Pool QC           2042
dtype: int64

In [8]:
# renaming columns
train.columns= train.columns.str.replace(" ","_").str.lower()    

In [9]:
train.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

In [10]:
# function that returns a list from a dataframes columns.
def get_list(df,string):
    mask=df.columns.str.contains(string)
    return list(df.columns[mask])

# Cleaning all basement related variables

In [11]:
def check_fill(df,ref_col,comparing_col,filler):
    
    '''Checking columns with NaN values against a reference column where 
    NaN values means 0 or none and filling these columns'''
    
    
    num_of_null=df.loc[df[comparing_col].isnull(),:].shape[0]
    check_num_of_null=df.loc[df[comparing_col].isnull(),:][ref_col].shape[0]
    if check_num_of_null == num_of_null:
        df[comparing_col].fillna(filler,inplace=True)
        return print("NaN values for", str(comparing_col), "has been filled with",str(filler))
    else:
        return print("NaN for", "comparing_col", "values do not match")

In [12]:
#list of string bsmt cols where NaN values means no basement
bsmt_string_list=["bsmt_exposure","bsmtfin_type_2","bsmtfin_type_1","bsmt_qual","bsmt_cond"]

In [13]:
# identifying the row that casues bsmt_exposure to have 3 more null values then 
# the rest of basements where NaN values mean no basement.
train.loc[(train["bsmt_exposure"].isnull())
          &(train["bsmt_qual"].notnull())
          &(train["bsmt_cond"].notnull())
          &(train["bsmtfin_type_2"].notnull())
          &(train["bsmtfin_type_1"].notnull()),:][bsmt_string_list]


,bsmt_exposure,bsmtfin_type_2,bsmtfin_type_1,bsmt_qual,bsmt_cond
1456,NaN,Unf,Unf,Gd,TA
1547,NaN,Unf,Unf,Gd,TA
1997,NaN,Unf,Unf,Gd,TA


In [14]:
# checking and filling bsmtfin_sf_1.
check_fill(train,"bsmt_exposure","bsmtfin_sf_1",0)

NaN values for bsmtfin_sf_1 has been filled with 0


In [15]:
# checking and filling bsmtfin_sf_2.
check_fill(train,"bsmt_exposure","bsmtfin_sf_2",0)

NaN values for bsmtfin_sf_2 has been filled with 0


In [16]:
# checking and filling total_bsmt_sf.
check_fill(train,"bsmt_exposure","total_bsmt_sf",0)

NaN values for total_bsmt_sf has been filled with 0


In [17]:
# checking and filling bsmt_unf_sf.
check_fill(train,"bsmt_exposure","bsmt_unf_sf",0)

NaN values for bsmt_unf_sf has been filled with 0


In [18]:
# checking and filling bsmt_half_bath.
check_fill(train,"bsmt_exposure","bsmt_half_bath",0)

NaN values for bsmt_half_bath has been filled with 0


In [19]:
# checking and filling bsmt_full_bath.
check_fill(train,"bsmt_exposure","bsmt_full_bath",0)

NaN values for bsmt_full_bath has been filled with 0


In [20]:
# checking and filling bsmt_cond.
check_fill(train,"bsmt_exposure","bsmt_cond","None")

NaN values for bsmt_cond has been filled with None


In [21]:
# checking and filling bsmt_qual.
check_fill(train,"bsmt_exposure","bsmt_qual","None")

NaN values for bsmt_qual has been filled with None


In [22]:
# checking and filling bsmtfin_type_1.
check_fill(train,"bsmt_exposure","bsmtfin_type_1","None")

NaN values for bsmtfin_type_1 has been filled with None


In [23]:
# checking and filling bsmtfin_type_2.
check_fill(train,"bsmt_exposure","bsmtfin_type_2","None")

NaN values for bsmtfin_type_2 has been filled with None


In [24]:
train["bsmt_exposure"].value_counts()

No    1339
Av     288
Gd     203
Mn     163
Name: bsmt_exposure, dtype: int64

In [25]:
# replacing NaN for bsmt_exposure with No
train["bsmt_exposure"].fillna("No",inplace=True)

In [26]:
#checking dtype of filled cols.
train[get_list(train,"bsmt")].dtypes

bsmt_qual          object
bsmt_cond          object
bsmt_exposure      object
bsmtfin_type_1     object
bsmtfin_sf_1      float64
bsmtfin_type_2     object
bsmtfin_sf_2      float64
bsmt_unf_sf       float64
total_bsmt_sf     float64
bsmt_full_bath    float64
bsmt_half_bath    float64
dtype: object

# Cleaning all Garage related columns 

In [27]:
#list of garage cols that where NaN values means no garage
gar_string_list=["garage_qual","garage_cond","garage_type","garage_finish","garage_yr_blt"]

In [28]:
# identifying the row that casues garage_type to have 1 less null value then the 
# rest of the garages columns where a NaN value means there is no garage.
train.loc[(train["garage_yr_blt"].isnull())&
          (train["garage_qual"].isnull())&
          (train["garage_cond"].isnull())&
          (train["garage_finish"].isnull())&
          (train["garage_type"].notnull()),:][gar_string_list]

,garage_qual,garage_cond,garage_type,garage_finish,garage_yr_blt
1712,NaN,NaN,Detchd,NaN,NaN


In [29]:
# garage area mean of deteched garages. 
detchd_sf_mean=train.loc[train["garage_type"]=="Detchd",:]["garage_area"].mean()

In [30]:
# checking and filling garage_cond with mean of detchd garage_area
check_fill(train,"garage_type","garage_area",detchd_sf_mean)

NaN values for garage_area has been filled with 419.54392523364487


In [31]:
#checking which type of garage
train.loc[train["garage_cars"].isnull(),:][gar_string_list]

,garage_qual,garage_cond,garage_type,garage_finish,garage_yr_blt
1712,NaN,NaN,Detchd,NaN,NaN


In [32]:
# garage car mode of deteched garages. 
detchd_car_mode=train.loc[train["garage_type"]=="Detchd",:]["garage_cars"].mode()

In [33]:
# checking and filling garage_cond.
check_fill(train,"garage_type","garage_cars",detchd_car_mode)

NaN values for garage_cars has been filled with 0    2.0
dtype: float64


In [34]:
# checking and filling garage_cond.
check_fill(train,"garage_type","garage_cond","None")

NaN values for garage_cond has been filled with None


In [35]:
# checking and filling garage_cond.
check_fill(train,"garage_type","garage_finish","None")

NaN values for garage_finish has been filled with None


In [36]:
# checking and filling garage_cond.
check_fill(train,"garage_type","garage_yr_blt",0)

NaN values for garage_yr_blt has been filled with 0


In [37]:
# checking and filling garage_cond.
check_fill(train,"garage_type","garage_qual","None")

NaN values for garage_qual has been filled with None


In [38]:
# replacing NaN for garage_cond with 0
train["garage_type"].fillna("None",inplace=True)

In [39]:
#checking dtype of filled cols.
train[get_list(train,"garage")].dtypes

garage_type       object
garage_yr_blt    float64
garage_finish     object
garage_cars      float64
garage_area      float64
garage_qual       object
garage_cond       object
dtype: object

# Cleaning Lot Frontage

In [40]:
# filtering for NaN values in Lot frontage
train.loc[train["lot_frontage"].isnull(),:].head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
7,145,535154050,20,RL,NaN,12160,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,5,2010,COD,142000
8,1942,535353130,20,RL,NaN,15783,Pave,NaN,Reg,Lvl,...,0,0,NaN,MnPrv,Shed,400,6,2007,WD,112500
23,12,527165230,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,185000
27,1534,909277040,50,RL,NaN,11700,Pave,Grvl,IR1,HLS,...,0,0,NaN,NaN,NaN,0,7,2008,WD,198000


In [41]:
# #filling NaN values with 0
train["lot_frontage"].fillna(0,inplace=True)

In [42]:
train.isnull().sum().sort_values().tail(8)

garage_cars        1
mas_vnr_type      22
mas_vnr_area      22
fireplace_qu    1000
fence           1651
alley           1911
misc_feature    1986
pool_qc         2042
dtype: int64

In [43]:
def fill_null(df,list_to_fill,fill_with):
    """function that fills NaN values in columns from a list"""
    [df[i].fillna(fill_with,inplace=True) for i in list_to_fill]

In [44]:
# remaing NaN values cannot be deduced from the other datas.
remaing_null_strings=["mas_vnr_type","fireplace_qu","fence","alley","misc_feature","pool_qc"]

# filling non numeric column's NaN with None.
fill_null(train,remaing_null_strings,"None")

#filling numeric colum NaN with 0
fill_null(train,["mas_vnr_area","lot_frontage"],0)

In [45]:
# Checking for NaN values
train.isnull().sum().sort_values().tail()


roof_style        0
year_remod/add    0
exter_qual        0
saleprice         0
garage_cars       1
dtype: int64

# ADD GET DUMMIES SAVE ONE WITH AND ONE WITHOUT 

# Cleaning Test dataset

In [46]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [47]:
test.shape

(879, 80)

In [48]:
test.dtypes

Id                int64
PID               int64
MS SubClass       int64
MS Zoning        object
Lot Frontage    float64
                 ...   
Misc Feature     object
Misc Val          int64
Mo Sold           int64
Yr Sold           int64
Sale Type        object
Length: 80, dtype: object

In [49]:
test.columns=test.columns.str.replace(" ","_").str.lower()
test.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

In [50]:
test.shape

(879, 80)

In [51]:
test.isnull().sum().sort_values().tail(20)

overall_cond        0
electrical          1
mas_vnr_type        1
mas_vnr_area        1
bsmt_cond          25
bsmtfin_type_2     25
bsmt_qual          25
bsmtfin_type_1     25
bsmt_exposure      25
garage_type        44
garage_finish      45
garage_yr_blt      45
garage_qual        45
garage_cond        45
lot_frontage      160
fireplace_qu      422
fence             707
alley             821
misc_feature      838
pool_qc           875
dtype: int64

# Cleaning Basments NaN values

In [52]:
# identifying that all 25 rows in bsmt that have NaN are the same rows. 
test.loc[(test["bsmt_exposure"].isnull())
          &(test["bsmt_qual"].isnull())
          &(test["bsmt_cond"].isnull())
          &(test["bsmtfin_type_2"].isnull())
          &(test["bsmtfin_type_1"].isnull()),:][bsmt_string_list].shape


(25, 5)

In [53]:
#filling NaN values in all bsmt cols with None as they non numeric 
fill_null(test,bsmt_string_list,"None")

In [54]:
#checking dtypes
test[bsmt_string_list].dtypes

bsmt_exposure     object
bsmtfin_type_2    object
bsmtfin_type_1    object
bsmt_qual         object
bsmt_cond         object
dtype: object

# Cleaning Garage NaN values

In [55]:
garage_col_list=get_list(test,"garage")

In [56]:
#adding id to garage_col_list
garage_col_list.append("id")


# identifying the row that casues garage_type to have 1 less null value then the 
# rest of the garages columns where a NaN value means there is no garage.
test.loc[(test["garage_yr_blt"].isnull())&
          (test["garage_qual"].isnull())&
          (test["garage_cond"].isnull())&
          (test["garage_finish"].isnull())&
          (test["garage_type"].notnull()),:][garage_col_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,id
765,Detchd,NaN,NaN,1,360,NaN,NaN,1357


In [57]:
#creating df with all garage type == detchd and cars==1

test_gar_car_detchd=test.loc[(test["garage_cars"]==1)&(test["garage_type"]=="Detchd"),:][garage_col_list]

In [58]:
#finding out the mode of each col in garages.
test_gar_car_detchd.describe(include="all")

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,id
count,125,124.000000,124,125.0,125.000000,124,124,125.000000
unique,1,NaN,3,NaN,NaN,3,4,NaN
top,Detchd,NaN,Unf,NaN,NaN,TA,TA,NaN
freq,125,NaN,120,NaN,NaN,96,101,NaN
mean,NaN,1947.241935,NaN,1.0,298.616000,NaN,NaN,1581.600000
std,NaN,23.351806,NaN,0.0,100.182477,NaN,NaN,776.383219
min,NaN,1900.000000,NaN,1.0,160.000000,NaN,NaN,31.000000
25%,NaN,1926.000000,NaN,1.0,240.000000,NaN,NaN,942.000000
50%,NaN,1948.500000,NaN,1.0,280.000000,NaN,NaN,1545.000000
75%,NaN,1962.250000,NaN,1.0,326.000000,NaN,NaN,2056.000000


In [59]:
# test_gar_car_detchd["garage_finish"].value_counts()[0]

In [60]:
#filling row with NaN values for garage type,year built, finish, quality and condition with either the mean or mode from the train dataset. 
train_gar_car_detchd=train.loc[(train["garage_cars"]==1)&(train["garage_type"]=="Detchd"),:][garage_col_list]


test.loc[test["id"]==1357,"garage_finish"]=train_gar_car_detchd["garage_finish"].value_counts()[0]
test.loc[test["id"]==1357,"garage_qual"]=train_gar_car_detchd["garage_qual"].value_counts()[0]
test.loc[test["id"]==1357,"garage_cond"]=train_gar_car_detchd["garage_cond"].value_counts()[0]
test.loc[test["id"]==1357,"garage_yr_blt"]=train_gar_car_detchd["garage_yr_blt"].mean()

## COMMENT
Because the numbers of varibles imputed are very low less then 1%, data leakage would be significant. As compared to if I were to impute NaN values with mean or mode of features, without doing a train test split in the start, that would lead into a more significant data leakage. 

In [61]:
#checking 
test.loc[test["id"]==1357,:][garage_col_list]

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,id
765,Detchd,1949.713147,244,1,360,202,218,1357


In [62]:
# identifying the row that casues garage_type to have 1 less null value then the 
# rest of the garages columns where a NaN value means there is no garage.
test.loc[(test["garage_yr_blt"].isnull())&
          (test["garage_qual"].isnull())&
          (test["garage_cond"].isnull())&
          (test["garage_finish"].isnull())&
          (test["garage_type"].isnull()),:][gar_string_list].shape

(44, 5)

In [63]:
fill_null(test,["garage_qual","garage_cond","garage_finish","garage_type"],"None")
fill_null(test,["garage_yr_blt"],0)

In [64]:
#checking dtypes
test[garage_col_list].dtypes

garage_type       object
garage_yr_blt    float64
garage_finish     object
garage_cars        int64
garage_area        int64
garage_qual       object
garage_cond       object
id                 int64
dtype: object

# Cleaning electrical 

In [65]:
#exploring electrical 
test["electrical"].describe(include="all")


count       878
unique        4
top       SBrkr
freq        814
Name: electrical, dtype: object

In [66]:
# assigning the mode of of electrical 
mode_elect=test["electrical"].describe(include="all")[2]
mode_elect

'SBrkr'

In [67]:
#filling the NaN in electrical with mode.
test["electrical"].fillna(mode_elect,inplace=True)

In [68]:
#checking dtypes
test["electrical"].dtypes

dtype('O')

# Cleaning the remaining of the cols 

In [69]:
# remaing NaN values cannot be deduced from the other datas.
remaing_null_strings=["mas_vnr_type","fireplace_qu","fence","alley","misc_feature","pool_qc"]

# filling non numeric column's NaN with None.
fill_null(test,remaing_null_strings,"None")

#filling numeric colum NaN with 0
fill_null(test,["mas_vnr_area","lot_frontage"],0)

In [70]:
#checking for NaN values
test.isnull().sum().sort_values().tail()

roof_matl         0
roof_style        0
year_remod/add    0
yr_sold           0
sale_type         0
dtype: int64

In [71]:
#checking dtypes
test[remaing_null_strings].dtypes

mas_vnr_type    object
fireplace_qu    object
fence           object
alley           object
misc_feature    object
pool_qc         object
dtype: object

In [72]:
#checking dtypes
test[["mas_vnr_area","lot_frontage"]].dtypes

mas_vnr_area    float64
lot_frontage    float64
dtype: object

In [73]:
test.describe(include="all") 

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
count,879.000000,8.790000e+02,879.000000,879,879.000000,879.000000,879,879,879,879,...,879.000000,879.000000,879.000000,879,879,879,879.000000,879.000000,879.000000,879
unique,NaN,NaN,NaN,6,NaN,NaN,2,3,4,4,...,NaN,NaN,NaN,3,5,4,NaN,NaN,NaN,10
top,NaN,NaN,NaN,RL,NaN,NaN,Pave,None,Reg,Lvl,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,WD
freq,NaN,NaN,NaN,675,NaN,NaN,874,821,564,790,...,NaN,NaN,NaN,875,707,838,NaN,NaN,NaN,755
mean,1445.588168,7.165050e+08,58.270762,NaN,56.955631,10340.920364,NaN,NaN,NaN,NaN,...,2.594994,14.813424,1.882821,NaN,NaN,NaN,48.443686,6.207053,2007.824801,NaN
std,850.717105,1.889135e+08,42.211389,NaN,34.338680,10047.335167,NaN,NaN,NaN,NaN,...,24.948416,52.975963,29.899698,NaN,NaN,NaN,549.858353,2.644097,1.327396,NaN
min,2.000000,5.263021e+08,20.000000,NaN,0.000000,1477.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN
25%,692.500000,5.284861e+08,20.000000,NaN,40.000000,7298.500000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN
50%,1435.000000,5.354542e+08,50.000000,NaN,60.000000,9453.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN
75%,2197.000000,9.071921e+08,70.000000,NaN,78.500000,11606.500000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN


In [74]:
train.head()["saleprice"]

0    130500
1    220000
2    109000
3    174000
4    138500
Name: saleprice, dtype: int64

In [75]:
# get list of all object in train 
train_object_list=[]
for i in train.columns:
    if train[i].dtypes == object and i != "saleprice":
        train_object_list.append(i)
# get list of all object in test
test_object_list=[]
for i in test.columns:
    if test[i].dtypes == object:
        test_object_list.append(i)


In [76]:
test[test_object_list].describe(include="all")[1:2]

,ms_zoning,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,...,fireplace_qu,garage_type,garage_finish,garage_qual,garage_cond,paved_drive,pool_qc,fence,misc_feature,sale_type
unique,6,2,3,4,4,2,5,3,26,9,...,6,7,5,6,7,3,3,5,4,10


In [77]:
train[train_object_list].describe(include="all")[1:2]

,ms_zoning,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,...,fireplace_qu,garage_type,garage_finish,garage_qual,garage_cond,paved_drive,pool_qc,fence,misc_feature,sale_type
unique,7,2,3,4,4,3,5,3,28,9,...,6,7,4,6,6,3,5,5,6,9


In [78]:
test_id=test[["id"]]

In [79]:
train.to_csv("../datasets/train_c.csv")
test.to_csv("../datasets/test_c.csv")
test_id.to_csv("../datasets/test_id.csv")